
# PARLA

## Problem
- Write a function to estimate the required minimal sample size to test the hypothesis of equal means
    - For more info see function's docstring below
    - To calculate minimal sample size use formula:

$$
n > \frac{ [ {\Phi}^{-1}(1 - \alpha/2) + {\Phi}^{-1}(1 - \beta) ]^2 (\sigma^{2}_{X} + \sigma^{2}_{Y}) }{ \epsilon^2 }
$$

## Action
- To calculate the minimal sample size I:
    - calculated z-scores using alpha and beta
    - calculated effect size
    - calculated minimal sample size using the formula
    - calculated the average number of observations per user
    - adjusted the minimal sample size by dividing by the average number of observations per user

## Result
- The function is implemented and successfully passed all tests

## Learning
- I revised relevant Python and Pandas functionality
- I learned and applied the formula for estimating the sample size for a t-test
- I learned that it is possible to adjust the minimal sample size by dividing it by the average number of observations per user

## Application
- I can apply relevant Python and Pandas functionality for similar data-related problems
- I can estimate the minimal sample size for a t-test
- I can adjust the minimal sample size, using the average number of observations per user


In [2]:

import numpy as np
import pandas as pd
from scipy import stats


In [3]:

def estimate_sample_size(
    metrics: pd.DataFrame,
    effect: float,
    alpha: float,
    beta: float
) -> int:
    """
    Estimate the required minimal sample size to test the hypothesis of equal means.

    Notes:
    - Effect size is specified as a percentage.
    - Standard deviation should be computed using np.std with default parameters.
    - Do not perform aggregation inside the function.
    - Mean and standard deviation must be calculated directly from the metric column.

    For metrics with one value per user, the sample size is computed directly using formula.
    For metrics with multiple values per user (e.g., response_time),
    the required number of observations is adjusted by dividing by the average number
    of observations per user.

    Example:
    If 'metrics' contains 1000 observations for 100 unique users,
    and the required number of observations is 302,
    then the group size will be 31 users (because 1000/100 = 10 obs/user,
    and 302 / 10 = 30.2, rounded up to 31).

    :param metrics: DataFrame with columns ['user_id', 'metric'].
    :param effect: Expected effect size (e.g., 3 for 3% increase of the mean value).
    :param alpha: Significance level.
    :param beta: Probability of Type II error (1 - power).
    :return: Required sample size (number of users per group).
    """

    # calculate z-scores using alpha and beta
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = stats.norm.ppf(1 - beta)

    # calculate effect size
    effect_percents = effect / 100
    e = np.mean(metrics.metric) * effect_percents

    # calculate minimal sample size using formula
    n = (z_alpha + z_beta)**2 * (np.std(metrics.metric)**2 * 2) / e**2

    # calculate average number of observations per user
    observations_per_user = metrics.groupby(['user_id'])['metric'].count().mean()

    # adjust the minimal sample size by dividing by the average number of observations per user
    # if there is only 1 observation per user, nothing will change
    # otherwise, the minimal sample size will be adjusted
    n = n / observations_per_user

    return np.ceil(n).astype(int)




In [4]:

# test_01
metrics = pd.DataFrame({
    'user_id': np.arange(100),
    'metric': np.linspace(500, 1490, 100)
})
effect, alpha, beta = 3, 0.05, 0.1
sample_size = estimate_sample_size(metrics, effect, alpha, beta)
print(f'test_01: {sample_size == 1966}')

# test_02
metrics = pd.DataFrame({
    'user_id': np.arange(100) % 30,
    'metric': np.linspace(500, 1490, 100)
})
effect, alpha, beta = 3, 0.05, 0.1
sample_size = estimate_sample_size(metrics, effect, alpha, beta)
print(f'test_02: {sample_size == 590}')


test_01: True
test_02: True
